In [38]:
import pickle
from sklearn.pipeline import make_pipeline
import lightgbm

In [47]:

MODEL_PATH = r"worker/models/forest_model"
loaded_model_log_reg = pickle.load(open(f"{MODEL_PATH}/forest_model.pkl", "rb"))


In [3]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [4]:
import pandas as pd
from worker.models.data_models.elderly_people import DataModel

In [48]:
data_dict = {
    "temperature": 20.202778,
    "humidity": 49.903167,
    "CO2CosIRValue": 81.994444,
    "CO2MG811Value": 618.461111,
    "MOX1": 427.705556,
    "MOX2": 757.966667,
    "COValue": 111.450000,
    "hour": 14
}
d = DataModel(**data_dict)
data_list = [20.337611, 57.908111, 43.650000, 518.077778, 529.527778, 702.811111, 150.288889, 10]
a = DataModel(**dict(zip(DataModel.__annotations__.keys(), data_list)))

In [49]:
data = pd.DataFrame([a.model_dump()], columns=['temperature','humidity','CO2CosIRValue','CO2MG811Value', 'MOX1','MOX2','COValue','hour'])

In [51]:
loaded_model_log_reg

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['hour']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['temperature', 'humidity',
                                                   'CO2CosIRValue',
                                                   'CO2MG811Value', 'MOX1',
                                                   'MOX2', 'COValue'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=2,
                                        random_state=42))])

In [6]:
from worker.models.log_reg_model import LogisticRegression

In [16]:
reg = LogisticRegression("log_reg_model", unload_interval=10)

In [26]:
res = reg.predict(data_dict)

In [27]:
res

array([0])

In [29]:
reg.model

In [30]:
from api.db_model import get_session

In [34]:
from datetime import MAXYEAR, MINYEAR, datetime
from api.db_model import TransactionHistory
from sqlalchemy import select

async for session in get_session():
    start_date = datetime(MINYEAR, 1, 1)
    end_date = datetime(MAXYEAR, 12, 31)
    query = (
        select(TransactionHistory)
        .filter(TransactionHistory.user_id == "7b5ff707-56b6-48fb-9159-66f63e35431f")
        .filter(TransactionHistory.timestamp >= start_date)
        .filter(TransactionHistory.timestamp <= end_date)
        .order_by(TransactionHistory.timestamp.asc())
    )
    result = await session.execute(query)

    transactions = result.scalars().all()

gaierror: [Errno -3] Temporary failure in name resolution

In [15]:
import pandas as pd
from datetime import datetime
import streamlit as st

# Ваш JSON
data = [
    {
        "user_id": "6531f0ee-2a2e-4e97-a771-cc4de9011aeb",
        "amount": 100,
        "result": {"message": "Trial credit of 100"},
        "err_reason": None,
        "model_id": None,
        "job_id": "59824bef-80db-485c-88af-abb3c767be91",
        "status": "IN_PROGRESS",
        "timestamp": "2024-01-25T01:32:15.413379+00:00",
    },
    {
        "user_id": "6531f0ee-2a2e-4e97-a771-cc4de9011aeb",
        "amount": -5,
        "result": "{\"data\": {\"temperature\": 0.0, \"humidity\": 0.0, \"CO2CosIRValue\": 0.0, \"CO2MG811Value\": 0.0, \"MOX1\": 0.0, \"MOX2\": 0.0, \"COValue\": 0.0, \"hour\": 0}, \"result\": [0]}",
        "err_reason": None,
        "model_id": 1,
        "job_id": "f0920465-82a0-42c5-b42a-4973d9c6a16e",
        "status": "SUCCESS",
        "timestamp": "2024-01-25T01:33:42.401933+00:00",
    },

]

# Преобразование timestamp в формат даты и время
for item in data:
    item["timestamp"] = datetime.strptime(item["timestamp"], "%Y-%m-%dT%H:%M:%S.%f%z")


# Создание датафрейма
df = pd.DataFrame(data)

# Определение временных границ
min_timestamp = df["timestamp"].min()
max_timestamp = df["timestamp"].max()

# Определение частоты
num_intervals = 10
freq = (max_timestamp - min_timestamp) / num_intervals

# Группировка и агрегация по динамической частоте
df_grouped = (
    df.groupby(pd.Grouper(key="timestamp", freq=freq))
    .agg({"amount": lambda x: x[x > 0].sum()})
    .rename(columns={"amount": "zachisleno"})
    .join(
        df.groupby(pd.Grouper(key="timestamp", freq=freq))
        .agg({"amount": lambda x: x[x < 0].sum()})
        .rename(columns={"amount": "potracheno"})
    )
    .fillna(0)
)

# Превращение индекса в колонку
df_grouped = df_grouped.reset_index()


# Визуализация с использованием Streamlit


In [16]:
df_grouped

,timestamp,zachisleno,potracheno
0,2024-01-25 01:32:12.472034400+00:00,100,0
1,2024-01-25 01:32:21.170889800+00:00,0,0
2,2024-01-25 01:32:29.869745200+00:00,0,0
3,2024-01-25 01:32:38.568600600+00:00,0,0
4,2024-01-25 01:32:47.267456+00:00,0,0
5,2024-01-25 01:32:55.966311400+00:00,0,0
6,2024-01-25 01:33:04.665166800+00:00,0,0
7,2024-01-25 01:33:13.364022200+00:00,0,0
8,2024-01-25 01:33:22.062877600+00:00,0,0
9,2024-01-25 01:33:30.761733+00:00,0,0
